<h1> Ensemble generation submission and analysis notebook </h1>

This script uses a combination of bash and python cells to form a workflow for creating and analysing FATES (or any CLM) ensembles. It will:

<ol>
<li>Clone the github repo 
<li>Make and build a default 4x5 CLM-FATES case
<li>Make an ensemble of CLM-FATES cases 
<li>Generate an ensemble of parameter files with one at a time or latin hypercube  modification
<li>Point each ensemble member at a different parameter file
<li>Submit all these jobs
<li>Analyse output
</ol>

<h4>n.b. This notebook is also set up to work with Cheyenne specific paths, but can be modified. 
<h4>n.b. The version here is relatively simple in that it just submits a one at a time sensemble and plots maps of the output. Obviously much more can be made of the ensemble design and analysis but this is here as a generic tool... 

#### Control variables for the analysis script.

In [1]:
# This mega script does 
# ensemble setup (including parameter file generation.  
# job submission and 
# analysis. 

# You can turn these off and on according to where you are tin the process. 

dosetup = 1 #do we want to create parameters files and so on?
dosubmit = 1 #do we do the submission stage, or just the analysis?
forcenewcase = 0 #do we scurb all the old cases and start again?
doanalysis = 1 #do we want to plot the outputs? 
print("setup, submit, analysis:", dosetup,dosubmit,doanalysis)

setup, submit, analysis: 1 1 1


#### Define parameter space for senstivity analysis. 

In [2]:
#aspiration to make this into a dictionary...
parameter_list=['fates_rad_leaf_clumping_index','fates_rad_leaf_xl',
                'fates_rad_leaf_rhovis','fates_rad_stem_rhovis',
                'fates_rad_leaf_tauvis','fates_rad_stem_tauvis' ,
               'fates_vai_top_bin_width','fates_vai_width_increase_factor']
nparams = 8
#how many members are in the ensemble?
ncases=nparams*2
min_delta=[0.5,   0.01,0.75, 0.75, 0.75,0.75, 0.25, 1.1]
max_delta=[1/0.85,1.4 ,1.25, 1.25, 1.25,1.25, 0.5, 1.25]
print(parameter_list)  

['fates_rad_leaf_clumping_index', 'fates_rad_leaf_xl', 'fates_rad_leaf_rhovis', 'fates_rad_stem_rhovis', 'fates_rad_leaf_tauvis', 'fates_rad_stem_tauvis', 'fates_vai_top_bin_width', 'fates_vai_width_increase_factor']


<h3>Set global variables and paths for ensemble analysis </h3>

In [3]:
# FATES mode settings for the ensemble
sp=1 #are we using SP mode?
calib=1 # this script can be run in one at a time and latin hypercube mode.
calibvars=2 #which set of variables are we peturbing?
n=1 #the number of the ensemble. For when you need to do it all again ;) 
dim3=0

### Set pathnames for parameter directories and casenames.

#### Import libraries

In [4]:
import os
import netCDF4 as nc4
import sys
import shutil
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import xarray as xr
from matplotlib import pyplot as plt
#import datetime

In [5]:
# what is your clone of the ctsm repo called? (or you want it to be called?) 
ctsmrepo='ctsmsep22'

#what do you want the directory with the ensemble cases in to be called?
ens_directory='RTM_ens'+str(n)

#what do you want the case names to begin with?
caseroot=ens_directory+'_case_'

USER='rfisher'

#path to scratch (or where the model is built.)
defbuildroot='/glade/scratch/'

#path to scratch (or where the model is built.)
output_dir='/glade/scratch/'+USER+'/'

#where are we now?
notebookdr=os.getcwd() 

# git directory. aka where do you want the code and scripts to live?
gitpath = '/glade/work/'+USER+'/'

scriptsdir='/glade/work/'+USER+'/git/'+ctsmrepo+'/cime/scripts/'




#### SP/NOCOMP specific paths to default case/param files

In [6]:
if(sp == 1): #Settings for the SP ensemble.
    resub=0 #how many times to resubmit the model. 

    #what is the name of your 'basecase'?
    defcase='fates_crops_smo_minlai_opt_vcmax50'
    
    #what is the base parameter file called?
    paramfiledefault='fates_params_default.nc'    

else: #Settings for the NOCOMP ensemble. 
    resub=2 #how many times do we want to resubmit the job?
    
    #what is the name of your 'basecase'?
    defcase='SPdefault_sept'
    
    #what is the base parameter file called?
    paramfiledefault='fates_params_default.nc'   


RTM_ens1_case_


#### Make a function that creates directories if they don't already exist

In [7]:
def make_directory(fileroot):   
    if(os.path.isdir(fileroot)):
        print('dir exists:'+fileroot)
    else:
        os.mkdir(fileroot)
        print('made: '+fileroot)

#### Make a git repo if needed. 

In [8]:
make_directory(gitpath+'git')

dir exists:/glade/work/rfisher/git


#### Clone CTSM repo (if necessary). BASH

##### (This box and others like it go into BASH script 'mode', and hence isn't in python) 

In [21]:
%%bash -s "$ctsmrepo" "$dosetup"
ctsmrepo=$1
dosetup=$2

if [ $dosetup -eq 1 ]
then
cd /glade/work/$USER/git
#go to git directory
pwd
#go to git repo, or checkout code
if [[ -d "$ctsmrepo" ]]
then
cd $ctsmrepo
echo "Already have ctsm repo"
else
echo "Cloning ctsm"
#clone CTSM code if you didn't to this already. 
git clone https://github.com/escomp/ctsm $ctsmrepo
cd $ctsmrepo
./manage_externals/checkout_externals
cd src
fi
fi


/glade/work/rfisher/git
Already have ctsm repo


#### Make a default FATES 4x5 case BASH 

In [22]:
%%bash -s "$scriptsdir" "$dosetup" "$defcase" 
scriptsdir=$1
dosetup=$2
defcase=$3
echo "defcase is" $defcase
echo "setup=" $dosetup
echo $dosetup
compset=2000_DATM%GSWP3v1_CLM50%FATES_SICE_SOCN_MOSART_SGLC_SWAV_SESP
if [[ $dosetup -eq 1 ]]
then
cd $scriptsdir
if [[ -d "$defcase" ]]
then
    echo "$defcase exists on your filesystem." .$defcase
else
    echo "making defcase.",$defcase
./create_newcase --case $defcase --compset 2000_DATM%GSWP3v1_CLM50%FATES_SICE_SOCN_MOSART_SGLC_SWAV_SESP --res f45_f45_mg37  --run-unsupported
cd $defcase
./case.setup
./case.build
fi
fi
echo "Currently in" $(pwd)

defcase is fates_crops_smo_minlai_opt_vcmax50
setup= 1
1
fates_crops_smo_minlai_opt_vcmax50 exists on your filesystem. .fates_crops_smo_minlai_opt_vcmax50
Currently in /glade/work/rfisher/git/ctsmsep22/cime/scripts


#### Adjust settings of default case. BASH 

In [23]:
%%bash -s "$scriptsdir" "$defcase" "$dosetup" 

scriptsdir=$1
defcase=$2
dosetup=$3

if [[ $dosetup -eq 1 ]]
then
defbld=$defbuildroot$USER/$defcase/bld
cd $scriptsdir$defcase

echo 'updating settings'
./xmlchange CONTINUE_RUN=FALSE;  
./xmlchange --id STOP_N --val 5;  #number of years of simulation
./xmlchange --id STOP_OPTION --val nyears;
./xmlchange --id CLM_FORCE_COLDSTART --val on;
echo "Currently in" $(pwd)
fi

updating settings
Currently in /glade/work/rfisher/git/ctsmsep22/cime/scripts/fates_crops_smo_minlai_opt_vcmax50


#### Make ensemble and parameter directory

In [24]:
make_directory(scriptsdir+ens_directory)
make_directory(scriptsdir+ens_directory+'/parameter_files' )

dir exists:/glade/work/rfisher/git/ctsmsep22/cime/scripts/RTM_ens1
dir exists:/glade/work/rfisher/git/ctsmsep22/cime/scripts/RTM_ens1/parameter_files


#### Make ensemble of CLM cases BASH

In [40]:

%%bash -s "$scriptsdir" "$defcase" "$dosetup" "$ens_directory" "$caseroot" "$ncases" "$forcenewcase"

scriptsdir=$1
defcase=$2
dosetup=$3
ens_directory=$4
caseroot=$5
ncases=$6
forcenewcase=$7

echo "ncases=" $ncases
echo "caseroot=" $caseroot
echo "dosetup=" $dosetup
echo "forcenewcase=" $forcenewcase
echo "ens_directory=" $ens_directory

if [[ $dosetup -eq 1 ]]
then
cd $scriptsdir

counter1=0 #include a default zero case. 
while [ $counter1 -le $ncases ]
do
  newcase=$caseroot$counter1 #name of ensemble membr case.    
  if [ -d $ens_directory/$newcase ]
  then
    echo ' new case already exists',$ens_directory/$newcase
    if [ $forcenewcase -eq 1 ]
    then
    echo 'force making case', $ens_directory/$newcase
    rm -rf $ens_directory/$newcase
    ./create_clone --clone $defcase --case $ens_directory/$newcase ;
    cd $ens_directory/$newcase
    echo 'case setup', $ens_directory/$newcase
    ./case.setup;
     cd ../../ 
    fi    
  else
   echo 'making new case', $ens_directory/$newcase
    ./create_clone --clone $defcase --case $ens_directory/$newcase ;
    cd $ens_directory/$newcase
    echo 'case setup', $ens_directory/$newcase
    ./case.setup;
    cd ../../ 
  fi
((counter1++))
done

fi

ncases= 16
caseroot= RTM_ens1_case_
dosetup= 1
forcenewcase= 0
ens_directory= RTM_ens1
 new case already exists,RTM_ens1/RTM_ens1_case_0
 new case already exists,RTM_ens1/RTM_ens1_case_1
 new case already exists,RTM_ens1/RTM_ens1_case_2
 new case already exists,RTM_ens1/RTM_ens1_case_3
 new case already exists,RTM_ens1/RTM_ens1_case_4
 new case already exists,RTM_ens1/RTM_ens1_case_5
 new case already exists,RTM_ens1/RTM_ens1_case_6
 new case already exists,RTM_ens1/RTM_ens1_case_7
 new case already exists,RTM_ens1/RTM_ens1_case_8
 new case already exists,RTM_ens1/RTM_ens1_case_9
 new case already exists,RTM_ens1/RTM_ens1_case_10
 new case already exists,RTM_ens1/RTM_ens1_case_11
 new case already exists,RTM_ens1/RTM_ens1_case_12
 new case already exists,RTM_ens1/RTM_ens1_case_13
 new case already exists,RTM_ens1/RTM_ens1_case_14
 new case already exists,RTM_ens1/RTM_ens1_case_15
 new case already exists,RTM_ens1/RTM_ens1_case_16


<h3> Create .nc version of current defualt file. I FAILED ATT THIS. NEEDS INPUT..</h3>
You currently have to do this step manually, by doing this in the terminal:

<i> cd /glade/work/$USER/git/$ctsmrepo/src/fates/parameter_files/</i><br>
<i>ncgen -o fates_params_default.nc fates_params_default.cdl </i>

#### Copy default template parameter file to ensemble parameter directory

In [41]:
if(dosetup == 1): 
    paramsdir='/glade/work/'+USER+'/git/'+ctsmrepo+'/cime/scripts/'+ens_directory+'/parameter_files'
    defparamfile='/glade/work/'+USER+'/git/'+ctsmrepo+'/src/fates/parameter_files/'+paramfiledefault
    filename_template = paramsdir+'/'+paramfiledefault
    try:
        os.remove(filename_template)
    except:
        print('file does not yet exist: '+filename_template)
    shutil.copy(defparamfile,filename_template)
    print('default parameters in'+filename_template)

default parameters in/glade/work/rfisher/git/ctsmsep22/cime/scripts/RTM_ens1/parameter_files/fates_params_default.nc


#### Make a default user_nl_clm file

In [42]:
if (dosetup == 1 ):
    defcasedir = scriptsdir+'/'+defcase #where is the vanilla file?
    os.chdir(defcasedir) #go to the def case dir
    if(os.path.isfile('/user_nl_clm_store')):
        #move the stored version to the default version
        shutil.copy(user_nl_clm_store, user_nl_clm_default)
    else:
        #store the 'vanilla' user_nl_clm file
        shutil.copy('user_nl_clm', 'user_nl_clm_store')
        # and then make a copy of that to act as the default for the ensemble. 
        shutil.copy('user_nl_clm_store', 'user_nl_clm_default')
        
    #write configuration and dummy parameter filename into new file. 
    f = open('user_nl_clm_default', 'a')
    if (sp ==1): #change to SP mode. 
        f.write('use_fates_sp = true')
        f.write('\n')
    f.write('fates_paramfile = "nullparameterfile"')
    f.close()
    # copy this 'fresh' default file into the ensemble directory. 
    shutil.copy('user_nl_clm_default', scriptsdir+'/'+ens_directory+'/user_nl_clm_default')
    
    #check what you made!
    #os.system("cat user_nl_clm_default")
    os.chdir(notebookdr)
print('end')


end


#### Create ensemble of parameter files

In [43]:
if(dosetup == 1): 
    fatesparamfile= 'fates_params_'+ens_directory+'_'
    vs=range(0,ncases+1) 
    for i in vs:
        filename_out = paramsdir+'/'+fatesparamfile+str(i)+'.nc' 
        #print('make:'+filename_out)
        try:
            os.remove(filename_out)
        except:
            print('file does not yet exist: '+filename_out)
        shutil.copy(filename_template,filename_out)
print('end')

end


#### Make function to modify FATES parameter files. 

In [44]:
def def_pftfile( fileroot,i,variable,delta):   
    pfilename = fileroot+str(i)+'.nc' 
    fin = nc4.Dataset(pfilename, 'r+')
    print('modifying parameter file',i,variable,delta);
    var = fin.variables[variable]
    var[:] = var[:]*delta
    fin.close()

#### Modify parameter files (one at a time)

In [45]:
#n.b. I (RAF) have other scripts that do LHC sampling or n dimensional arrays. 
#just including this one at a time example here for simplicity. 

if(dosetup == 1): 
    fileroot =  paramsdir+'/'+fatesparamfile 
    if(sp == 1):
        for p in range(0,8):
            def_pftfile(fileroot,p*2+1,parameter_list[p],min_delta[p])
            def_pftfile(fileroot,p*2+2,parameter_list[p],max_delta[p])


modifying parameter file 1 fates_rad_leaf_clumping_index 0.5
modifying parameter file 2 fates_rad_leaf_clumping_index 1.1764705882352942
modifying parameter file 3 fates_rad_leaf_xl 0.01
modifying parameter file 4 fates_rad_leaf_xl 1.4
modifying parameter file 5 fates_rad_leaf_rhovis 0.75
modifying parameter file 6 fates_rad_leaf_rhovis 1.25
modifying parameter file 7 fates_rad_stem_rhovis 0.75
modifying parameter file 8 fates_rad_stem_rhovis 1.25
modifying parameter file 9 fates_rad_leaf_tauvis 0.75
modifying parameter file 10 fates_rad_leaf_tauvis 1.25
modifying parameter file 11 fates_rad_stem_tauvis 0.75
modifying parameter file 12 fates_rad_stem_tauvis 1.25
modifying parameter file 13 fates_vai_top_bin_width 0.25
modifying parameter file 14 fates_vai_top_bin_width 0.5
modifying parameter file 15 fates_vai_width_increase_factor 1.1
modifying parameter file 16 fates_vai_width_increase_factor 1.25


#### Point each ensemble script at different parameter file

In [50]:
if(dosetup == 1): 
    root= '/glade/work/'+USER+'/git/'+ctsmrepo+'/cime/scripts/'+ens_directory+'/'
    fatesparamfile='fates_params_'+ens_directory+'_'
    vs=range(0,ncases+1) 
    print('paramsdir=',paramsdir)
    for i in vs:
        pftfilename = paramsdir+'/'+fatesparamfile+str(i)+'.nc'    
        print(fatesparamfile+str(i)+'.nc')
        unlfile=root+caseroot+str(i)+'/'+'user_nl_clm'
        #print('unl='+unlfile)   
        fin = open(unlfile, "rt")     
        data = fin.read()   #read file contents to string   
        data = data.replace('pyton', 'python') #replace all occurrences of the required string        
        fin.close() #close the input file       
        fin = open(unlfile, "wt") #open the input file in write mode     
        fin.write(data) #overrite the input file with the resulting data       
        fin.close() #close the file

paramsdir= /glade/work/rfisher/git/ctsmsep22/cime/scripts/RTM_ens1/parameter_files
fates_params_RTM_ens1_0.nc
fates_params_RTM_ens1_1.nc
fates_params_RTM_ens1_2.nc
fates_params_RTM_ens1_3.nc
fates_params_RTM_ens1_4.nc
fates_params_RTM_ens1_5.nc
fates_params_RTM_ens1_6.nc
fates_params_RTM_ens1_7.nc
fates_params_RTM_ens1_8.nc
fates_params_RTM_ens1_9.nc
fates_params_RTM_ens1_10.nc
fates_params_RTM_ens1_11.nc
fates_params_RTM_ens1_12.nc
fates_params_RTM_ens1_13.nc
fates_params_RTM_ens1_14.nc
fates_params_RTM_ens1_15.nc
fates_params_RTM_ens1_16.nc


#### Check parameter files were correctly modifed

In [51]:
i=5
paramnum=2
pf= paramsdir+'/'+fatesparamfile+str(0)+'.nc' 
print(pf)
find = nc4.Dataset(pf, 'r+')
vardef=find.variables[parameter_list[paramnum]]
pf= paramsdir+'/'+fatesparamfile+str(i)+'.nc' 
finm = nc4.Dataset(pf, 'r+')
varmod=finm.variables[parameter_list[paramnum]]
print('actual ratio:',np.divide(varmod[:],vardef[:]))
print('target ratio:',min_delta[paramnum])

/glade/work/rfisher/git/ctsmsep22/cime/scripts/RTM_ens1/parameter_files/fates_params_RTM_ens1_0.nc
actual ratio: [0.75 0.75 0.75 0.75 0.75 0.75 0.75 0.75 0.75 0.75 0.75 0.75]
target ratio: 0.75


### Submit ensemble of cases BASH

In [52]:
%%capture
%%bash -s "$scriptsdir" "$defcase" "$ens_directory" "$caseroot" "$ncases" "$dosubmit" 

scriptsdir=$1
defcase=$2
ens_directory=$3
caseroot=$4
ncases=$5
dosubmit=$6

if [ $dosubmit -eq 1 ]
then
cd $scriptsdir

echo 'submitting'
startcase=0
counter1=$startcase
cd $ens_directory
pwd

while [ $counter1 -le $ncases ]
do
  echo $counter1
  newcase=$caseroot$counter1
  echo $newcase

  if [ -d $newcase ]
  then
    cd $newcase
    cat user_nl_clm
    echo 'submitting job',$newcase
    ./xmlchange BUILD_COMPLETE=TRUE
    #./xmlchange RESUBMIT=1
   # ./xmlchange PROJECT=P93300041
    ./case.submit
    cd ../
  else
   echo 'no case', $newcase
  fi

  ((counter1++))
  done
else
echo "not submitting jobs"
fi

<h2>Analyse output</h2>

### Paths for analysis

In [ ]:
if doanalysis ==1: 
    output='/glade/scratch/rfisher/'
    conv = 3600*24*365
    yr='.clm2.h0.'   
    ychoose=2001
    delta=1
    print('ychoose=', ychoose)

<h3>Make output figure directory</h3>

In [ ]:
figpath = (notebookdr+ 'figs_'+ens_directory)
make_directory(figpath)

<h3>Make a single data structure for the ensemble</h3>

In [ ]:
ychoose=2003
print(output_dir)
if doanalysis ==1:
    hstring='.clm2.h0.'
    vs=range(0,ncases+1)
    count=1
    print(caseroot)
    for i in vs:       
        run=caseroot+str(i)
        os.listdir(output_dir + '/archive')
        arc = output_dir + 'archive/' + run + '/lnd/hist/' 
        hpath = arc
        tfile = run+hstring+str(ychoose)+'-01.nc'
        print(hpath)
        if(os.path.isdir(hpath)): 
            print('is archive')
            if(os.path.isfile(hpath+tfile)): 
                print('file in archive')
            else:
                print('file not in archive',hpath+tfile)
                #os.listdir(hpath)
        else:
            print('no archive')
            hpath = output_dir + run + '/run/'            
            if(os.path.isdir(hpath)): 
                print('is rundir',hpath)                
                if(os.path.isfile(hpath+tfile)):
                    print('file in  rundir',hpath+tfile)
                else:
                    print('no file in  rundir',hpath+tfile)                    
            else:
                print('no  rundir',hpath)

        #print(hpath)
        vars=['FATES_GPP','FSR','SABV','lat','lon','time']
        print(hpath +run+hstring+str(ychoose)+'-01.nc')
        allvars=list(xr.open_dataset(hpath +run+hstring+str(ychoose)+'-01.nc', decode_times=False).variables)
        dropvars=list(set(allvars) - set(vars)) #thanks to Ben for figuring this part out :) 
        tmp=xr.open_mfdataset(arc +run+hstring+str(ychoose)+'*', decode_times=False, drop_variables=dropvars)

        if i==0:
            dsc = tmp           
        else:
            dsc=xr.concat([dsc,tmp],'ens')

print('end')

#### Save data in netcdf file just incase

In [ ]:
outfile=figpath+'/dataout.nc' 
dsc.to_netcdf(outfile)
os.listdir(figpath)

#### Plot delta from default

In [ ]:
import time
rel=0
v=3
if doanalysis ==1:
    conv=1
    fig = plt.figure()
    plt.rcParams['figure.figsize'] = [19,7]
    fig.subplots_adjust(hspace=0.3, wspace=0.2)
    vs=range(1,ncases+1)
    ncol=7
    nrow=2
    count=1
    var_0=dsc.SABV.isel(ens=0)
    defm=np.multiply(np.mean(var_0,0),conv)
    for i in vs:
        index=((count+1) % 2)*ncol + ((count+1) // 2)
        #print(index,count,(count+1) % 2,((count+1) // 2),(((count+1) // 2))-1)
        ax = fig.add_subplot(nrow, ncol, index)

        #ds0=dse1.isel(realization=i)
        var_i=dsc.SABV.isel(ens=i)
        mod=np.multiply(np.mean(var_i,0),conv)
        
        delt=mod-defm
        plt1=delt.plot(cmap='RdYlBu')
                    
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        if((count+1) % 2==0):
            ex=' -'
        else:
            ex=' +'
        ax.set_title(str(parameter_list[(((count+1) // 2))-1]+ex))
        count=count+1
        fig.canvas.draw()
        time.sleep(1) 
          

In [ ]:
import time
rel=0
v=2
if doanalysis ==1:
    conv=1
    fig = plt.figure()
    plt.rcParams['figure.figsize'] = [19,7]
    fig.subplots_adjust(hspace=0.3, wspace=0.2)
    vs=range(1,ncases+1)
    ncol=7
    nrow=2
    count=1
    var_0=dsc.FSR.isel(ens=0)
    defm=np.multiply(np.mean(var_0,0),conv)
    for i in vs:
        index=((count+1) % 2)*ncol + ((count+1) // 2)
        #print(index,count,(count+1) % 2,((count+1) // 2),(((count+1) // 2))-1)
        ax = fig.add_subplot(nrow, ncol, index)

        #ds0=dse1.isel(realization=i)
        var_i=dsc.FSR.isel(ens=i)
        mod=np.multiply(np.mean(var_i,0),conv)
        
        delt=mod-defm
        plt1=delt.plot(cmap='RdYlBu')
                    
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        if((count+1) % 2==0):
            ex=' +'
        else:
            ex=' -'
        ax.set_title(str(parameter_list[(((count+1) // 2))-1]+ex))
        count=count+1
        fig.canvas.draw()
        time.sleep(1) 
          